In [1]:
import pandas as pd
import numpy as np

In [31]:
df = pd.read_excel("./SELSP_simOpt/DoE/DoE - Results v5.xlsx")

def round_values(smax_str):
    values = [float(x) for x in smax_str.strip('[]').split(', ')]
    rounded_values = [f"{val:.1f}" for val in values]
    # return "[" + ", ".join(rounded_values) + "]"
    return rounded_values

def round_float(value):
    return np.round(value, 1)

# Define a custom function to create tuples
def create_tuples(row):
    return [(smin, smax) for smin, smax in zip(row['Smin'], row['Smax'])]


# Apply the custom function to the "Smax" column
df['Smax'] = df['Smax'].apply(round_values)
df['Smin'] = df['Smin'].apply(round_values)
df['Cost'] = df['Cost'].apply(round_float)
df['SL'] = df['SL'].apply(round_float)

# Create a new column containing tuples
df['(s, S)'] = df.apply(create_tuples, axis=1)

df_latex = df[[col for col in df.columns if col not in ['Smin', 'Smax'] ]].copy()
df_latex.columns = ['OrdemPad', 'OrdemEns', 'N', 'cv', 'rho', 'p', 'Algoritmo', 'Custo', 'SL', '(s, S)']

# Define a custom function to format float values as strings
def format_as_string(value):
    if isinstance(value, float):
        return f"{value:,.1f}".replace(',', 'T').replace('.', ',').replace('T', '.')
    elif isinstance(value, list):
        formatted_list = [f"({float(x[0]):,.1f}".replace(',', 'X').replace('.', ',').replace('X', '.') + \
                      f"; {float(x[1]):,.1f}".replace(',', 'X').replace('.', ',').replace('X', '.') + ')'
                     for x in value]
        return ", ".join(formatted_list)
        # return "[" + ", ".join(formatted_list) + "]"
        
    return str(value)





In [45]:
df_latex_print = df_latex.applymap(format_as_string)

# Select the first 10 rows
df_head = df_latex_print.head(15)

# Select the last 10 rows
df_tail = df_latex_print.tail(15)

# Create a row with empty values (or "...") to insert between the DataFrames
empty_row = pd.DataFrame({'OrdemPad': ['...']})

# Concatenate the selected rows into a single DataFrame
df_selected = pd.concat([df_head, empty_row, df_tail])

print(df_selected.to_latex(index=False, formatters={"name": str.upper}, na_rep=""))

\begin{tabular}{llllllllll}
\toprule
OrdemPad & OrdemEns &  N &  cv & rho &   p & Algoritmo &     Custo &    SL &                                             (s, S) \\
\midrule
     470 &        1 &  5 & 0,2 & 0,7 & 0,0 &        NM & 103.489,4 &  99,1 & (1.201,4; 1.399,2), (914,7; 1.658,8), (639,6; 1... \\
     395 &        2 & 20 & 0,2 & 0,9 & 0,1 &        GA & 169.395,4 &  99,5 & (10,6; 19,1), (44,2; 647,5), (136,9; 517,6), (1... \\
     376 &        3 & 20 & 0,2 & 0,8 & 0,0 &       ACO & 160.969,9 &  98,7 & (60,0; 270,0), (155,0; 215,0), (560,0; 635,0), ... \\
     312 &        4 & 10 & 0,5 & 0,8 & 0,1 &       ACO & 100.176,4 &  99,3 & (175,0; 365,0), (260,0; 455,0), (585,0; 715,0),... \\
     968 &        5 & 10 & 0,5 & 0,9 & 0,1 &       ACO & 105.675,8 &  99,7 & (435,0; 825,0), (120,0; 490,0), (655,0; 935,0),... \\
      64 &        6 & 20 & 0,2 & 0,9 & 0,0 &       ACO & 178.758,1 &  99,6 & (535,0; 700,0), (15,0; 105,0), (35,0; 155,0), (... \\
     465 &        7 &  5 & 0,1 & 0,9 

In [44]:
df.groupby(by=['algorithm']).agg({'Cost':['mean', 'std'],'SL':['mean', 'std'] }).round(2)

Cost               SL      
                mean       std   mean   std
algorithm                                  
ACO        133173.14  37525.93  97.48  3.07
GA         138236.79  37815.27  96.56  4.10
NM         115424.01  27708.92  98.91  1.62
RD         133588.31  35908.79  97.24  3.49

In [48]:
table = df.groupby(by=['p']).agg({'Cost':['mean', 'std'],'SL':['mean', 'std'] }).round(2)
print(table.to_latex(index=True, formatters={"name": str.upper}, na_rep=""))

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{2}{l}{Cost} & \multicolumn{2}{l}{SL} \\
{} &       mean &       std &   mean &   std \\
p    &            &           &        &       \\
\midrule
0.00 &  128612.01 &  34563.72 &  97.65 &  3.24 \\
0.05 &  129988.41 &  35364.88 &  97.50 &  3.43 \\
0.15 &  131716.26 &  38022.37 &  97.49 &  3.27 \\
\bottomrule
\end{tabular}



In [49]:
table = df.groupby(by=['N']).agg({'Cost':['mean', 'std'],'SL':['mean', 'std'] }).round(2)
print(table.to_latex(index=True, formatters={"name": str.upper}, na_rep=""))

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{2}{l}{Cost} & \multicolumn{2}{l}{SL} \\
{} &       mean &       std &   mean &   std \\
N  &            &           &        &       \\
\midrule
5  &  113443.56 &  41486.46 &  95.27 &  4.74 \\
10 &  112458.95 &  13878.13 &  98.54 &  1.25 \\
20 &  164414.18 &  14511.26 &  98.83 &  1.03 \\
\bottomrule
\end{tabular}



In [50]:
table = df.groupby(by=['cv']).agg({'Cost':['mean', 'std'],'SL':['mean', 'std'] }).round(2)
print(table.to_latex(index=True, formatters={"name": str.upper}, na_rep=""))

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{2}{l}{Cost} & \multicolumn{2}{l}{SL} \\
{} &       mean &       std &   mean &   std \\
cv   &            &           &        &       \\
\midrule
0.10 &  131018.56 &  38131.75 &  97.41 &  3.83 \\
0.25 &  129191.77 &  35346.70 &  97.53 &  3.33 \\
0.50 &  130106.36 &  34507.78 &  97.70 &  2.67 \\
\bottomrule
\end{tabular}



In [51]:
table = df.groupby(by=['rho']).agg({'Cost':['mean', 'std'],'SL':['mean', 'std'] }).round(2)
print(table.to_latex(index=True, formatters={"name": str.upper}, na_rep=""))

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{2}{l}{Cost} & \multicolumn{2}{l}{SL} \\
{} &       mean &       std &   mean &   std \\
rho &            &           &        &       \\
\midrule
0.7 &  148600.31 &  28440.41 &  95.73 &  4.62 \\
0.8 &  126623.52 &  29031.93 &  97.84 &  2.26 \\
0.9 &  115092.85 &  40777.61 &  99.07 &  0.86 \\
\bottomrule
\end{tabular}



In [52]:
table = df.groupby(by=['algorithm']).agg({'Cost':['mean', 'std'],'SL':['mean', 'std'] }).round(2)
print(table.to_latex(index=True, formatters={"name": str.upper}, na_rep=""))

\begin{tabular}{lrrrr}
\toprule
{} & \multicolumn{2}{l}{Cost} & \multicolumn{2}{l}{SL} \\
{} &       mean &       std &   mean &   std \\
algorithm &            &           &        &       \\
\midrule
ACO       &  133173.14 &  37525.93 &  97.48 &  3.07 \\
GA        &  138236.79 &  37815.27 &  96.56 &  4.10 \\
NM        &  115424.01 &  27708.92 &  98.91 &  1.62 \\
RD        &  133588.31 &  35908.79 &  97.24 &  3.49 \\
\bottomrule
\end{tabular}

